# Weather in Tromsø over time

## Hypothesis

In this project we will look at the historical weather data gathered in Tromsø and see if the winters and summers have changed in the last years compared to earlier trends. We will look at snow-depth, temperature, precipitation and other factors if there has been a changing trend and if one can see the effects of climate change in Tromsø. 

## Requirements

In [1]:
import pandas as pd 
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import altair as alt 
import reading_data as rd



In [89]:
#Midlertidig løsning for å oppdatere reading_data
import importlib
importlib.reload(rd)


<module 'reading_data' from '/Users/tha222/git-folder/troms_weather/scripts/reading_data.py'>

In [4]:
client_id = '../ignore_me/client_id.txt'
client_secret = '../ignore_me/client_secret.txt'
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## Finding the weather stations of Tromsø

Where ate the stations we have? And how old are they? 

In [ ]:
station_id = pd.DataFrame(rd.FindingStations(client_id, client_secret
                                             ).get_sensors(69.95, 18.95, 30, "TROMSØ"),
                                             columns=['station_id', 'name'])

start_date, coor, names = [], [], []
for station in station_id['station_id']:
    location = rd.ReadingData(station, client_id, client_secret)
    start, end, coordinates, name = location.get_station_info()
    start_date.append(start)
    coor.append(coordinates)
    names.append(name)


In [ ]:
station_id['date'] = start_date
station_id['year'] = pd.to_datetime(station_id['date']).dt.strftime('%Y')
station_id['lon'] = [cor[0] for cor in coor]
station_id['lat'] = [cor[1] for cor in coor]
station_id['name'] = names
station_id.sort_values(by='date', inplace=True)


In [ ]:
station_id.reset_index(drop=True, inplace=True)
station_id


In [ ]:
station_coordinates = gpd.GeoDataFrame(station_id, 
                                       geometry=gpd.points_from_xy(
                                           station_id.lon, station_id.lat))
station_coordinates.crs = 'EPSG:4326'
station_coordinates.head(20)


In [ ]:
gdf = gpd.read_file('../data/Kommuner-L.geojson')
hoyde = gpd.read_file('../data/Basisdata_55_Troms_25833_N1000Hoyde_GML.gml', 
                      layer='Høydekurve')
hoyde = hoyde.to_crs(gdf.crs)
tromso = gdf[gdf['name'] == 'Tromsø']
clipped_hoyde = gpd.clip(hoyde, tromso)
fig, ax = plt.subplots(figsize=(10, 10))
tromso.plot(ax=ax, color="linen", edgecolor="grey")
clipped_hoyde.plot(ax=ax, color="xkcd:taupe", markersize=0.1, alpha=0.5)
station_coordinates.plot(ax=ax, column='year', cmap='Accent', marker='o', legend=True, 
          alpha=0.7, edgecolor='indigo')
ax.set_xlim(18.1,19.7)
ax.set_ylim(69.52,70.05)
plt.show()


In [ ]:
chosen_stations = station_coordinates.iloc[[0, 1, 3, 4, 6]]
chosen_stations


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
tromso.plot(ax=ax, color="linen", edgecolor="grey")
clipped_hoyde.plot(ax=ax, color="xkcd:taupe", markersize=0.1, alpha=0.5)
station_coordinates.plot(ax=ax, color='lightgrey', marker='o', legend=True, 
          alpha=0.7, edgecolor='indigo')
chosen_stations.plot(ax=ax, color='Blue', marker='o', legend=True)
ax.set_xlim(18.1,19.7)
ax.set_ylim(69.52,70.05)
plt.show()


## Temperature over the years

How has temperature changed over time? 
- Average temperature for each season 
- Extreme temperature now and then 
- Daily temperature range over time for summer and winter 
- Local differences in temperature 


### MIDLERTIDIG START

Fokuserer på disse stasjonene:
- Kjølen
- Måsvik
- Værvarslinka
- Tromsø LH
- Oldervik 
  
for en blanding av fjell, by, mot hav og fjord

Vil bruke Værvarslinga for å se på large scale climate trends og de over til å se etter local environmental influences. 


### MIDLERTIDIG OVER

In [5]:
vervarslinga = rd.ReadingData('SN90450', client_id, client_secret)
temperature_yearly = vervarslinga.get_temperature_yearly('1885','2024')
temperature_yearly.head()


,year,min,max,mean
0,1921,-12.8,NaN,2.4
1,1922,-12.7,NaN,3.0
2,1923,-13.8,NaN,2.1
3,1924,-14.3,NaN,2.8
4,1925,-14.3,24.4,2.9


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(temperature_yearly['year'], temperature_yearly['mean'], linestyle='-', color='green')
plt.fill_between(temperature_yearly['year'], temperature_yearly['min'], temperature_yearly['max'], color='green', alpha=0.2)

# Labels and title
plt.xlabel("Year")
plt.ylabel("Temperature (°C)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
#Getting all the temperature data for the spring
spring_temperature = pd.DataFrame()
for year in range(1925, 2025):
    spring_temperature = pd.concat([spring_temperature, vervarslinga.get_temperature_daily(str(year)+'-03-01', str(year)+'-05-31')])


In [ ]:
#Getting all the summer data
summer_temperature = pd.DataFrame()
for year in range(1925, 2025):
    summer_temperature = pd.concat([summer_temperature, vervarslinga.get_temperature_daily(str(year)+'-06-01', str(year)+'-08-31')])


In [ ]:
#Getting all the atumn data
atumn_temperature = pd.DataFrame()
for year in range(1925, 2025):
    atumn_temperature = pd.concat([atumn_temperature, vervarslinga.get_temperature_daily(str(year)+'-09-01', str(year)+'-11-30')])


In [ ]:
#Getting all the winter data
winter_temperature = pd.DataFrame()
for year in range(1925, 2025):
    winter_temperature = pd.concat([winter_temperature, vervarslinga.get_temperature_daily(str(year)+'-12-01', str(year+1)+'-02-28')])


Takes like 4 mins to do, obs obs

In [ ]:
for frames in [spring_temperature, summer_temperature, atumn_temperature, winter_temperature]:
    frames['date'] = pd.to_datetime(frames['date'])
    frames['year'] = frames['date'].dt.year
    frames['month_day'] = frames['date'].dt.strftime("%m-%d")


In [ ]:

summer_heatmap = alt.Chart(summer_temperature).mark_rect().encode(
    x=alt.X("year:O", title="Year", axis=alt.Axis(labelAngle=-45)),  # Convert to ordinal for better visualization
    y=alt.Y("month_day:N", title="Day of Summer", sort=None),  # Use Month-Day format
    color=alt.Color("temperature:Q", scale=alt.Scale(scheme="reds", domain=[10, 25]), title="Avg Temp (°C)"),
    tooltip=["year", "month_day", "temperature"]
).properties(
    width=900,
    height=500,
    title="Summer Temperature Heatmap Over 99 Years"
)

summer_heatmap.show()


In [ ]:
winter_heatmap = alt.Chart(winter_temperature).mark_rect().encode(    
    x=alt.X("year:O", title="Year", axis=alt.Axis(labelAngle=-45)),  # Convert to ordinal for better visualization
    y=alt.Y("month_day:N", title="Day of Summer", sort=None),  # Use Month-Day format
    color=alt.Color("temperature:Q", scale=alt.Scale(scheme="blues", domain=[0, -20]), title="Avg Temp (°C)"),
    tooltip=["year", "month_day", "temperature"]
).properties(
    width=900,
    height=500,
    title="Winter Temperature Heatmap Over 99 Years"
)

winter_heatmap.show()
    

In [6]:
amomaly_temperature = vervarslinga.get_temperature_anomaly('1925', '2024')
amomaly_temperature.head()


,year,month,anomaly,year_month
0,1925,3,-0.6,1925-03
1,1925,6,1.0,1925-06
2,1925,9,-0.8,1925-09
3,1925,12,-0.1,1925-12
4,1926,3,-0.4,1926-03


In [48]:
color_change = alt.datum.anomaly > 0

alt.Chart(amomaly_temperature,
title=alt.Title("Temperatue change over time",
subtitle='Temperature Anomaly (°C compared to the 1960-1991 normal')
).mark_bar(
    bandSize=40,
).encode(
    x = alt.X("year_month:O", axis=alt.Axis(labelAngle=0,
                                            labelExpr='substring(datum.value, 0, 4)',
                                            values=['1925-03', '1935-03', '1945-03', '1955-03', '1965-03', '1975-03', '1985-03', '1995-03', '2005-03', '2015-03', '2024-03'],
                                            labelOverlap=True, ticks=False), title="Year"),
    y =alt.Y('anomaly:Q', title="Temperature Anomaly (°C)"),
#    tooltip=['year', 'month', 'anomaly'],
    color=alt.Color('anomaly:Q',  # Use 'value' for coloring
                    scale=alt.Scale(scheme='redblue', domain=[10, -10])), # Legend customization
).properties(
    width=600,
    height=400,
)


alt.Chart(...)